# Расчёт юнит экономики привлечения клиентов из разных каналов
Бизнесу необходимо произвести рассчёты эффективности вложения средств в разные каналы привлечения клиентов.

## Цели расчётов:
1. Определить какие каналы окупаются;
2. Определить какой из каналов самый прибыльный с точки зрения прибыли на один юнит;
3. Сформулировать бизнес выводы на основании данных расчётов.

## Ход расчётов:
Данные о LTV пользователей и стоимости привлечения находятся в файлах `customers.csv` и `cost_attraction.csv`. Перед выполнением расчётов необходимо произвести обзор данных. В предобработке данных будет произведена проверка на наличие ошибок в данных и при их наличии оценка их влияния на рассчёты. После этого будут осуществлены расчёты и формирование бизнес выводов.

В файле `customers.csv` содержится статистика по клиентам, привлеченным из каналов.
Поля:
- id клиента - номер клиента;
- Канал привлечения - канал из которого пришел клиент;
- LTV клиента, руб. - сколько прибыли принёс клиент за всё время взаимодействия с компанией.

В файле `cost_attraction.csv` содержится информация о общей стоимости привлечения для каждого канала.
Поля:
- Канал привлечения - канал, на который выполнены расходы на привлечение;
- Стоимость привлечения, руб. - сколько всего было потрачено на привлечение в данном канале.

## Стадии проекта:
1. Обзор данных
2. Предобработка данных
3. Расчёты
4. Результаты расчётов

# 1. Обзор данных
### Обзор таблиц:

In [122]:
import pandas as pd
# сохраняем в df данные по LTV пользователей
df_customers = pd.read_csv('data/customers.csv', sep = ';')
# сохраняем в df данные по затратам на привлечение в каждом канале
df_cost_attr = pd.read_csv('data/cost_attraction.csv', sep = ';')

Обзор данных по пользователям из таблицы `customers.csv`

In [123]:
# выведем первые 5 пользователей
df_customers.head()

,id клиента,Канал привлечения,"LTV, руб."
0,1,Реклама в новостных СМИ,778.0
1,2,Реклама в новостных СМИ,132.0
2,3,Реклама у блогеров,22.0
3,4,Таргетная реклама,118.0
4,5,Реклама у блогеров,367.0


Обзор данных по стоимости привлечения в каналах из таблицы `cost_attraction.csv`

In [124]:
# выведем данные по стоимости
df_cost_attr

,Канал привлечения,"Стоимость привлечения, руб."
0,Таргетная реклама,100000
1,e-mail рассылки,25000
2,Реклама у блогеров,120000
3,Контекстная реклама,90000
4,Нативная реклама,70000
5,Реклама в новостных СМИ,60000


### Общая информация о таблицах:

In [125]:
# информация о таблице `customers.csv`
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id клиента         1800 non-null   int64  
 1   Канал привлечения  1800 non-null   object 
 2   LTV, руб.          1800 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 42.3+ KB


In [126]:
# информация о таблице `cost_attraction.csv`
df_cost_attr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Канал привлечения            6 non-null      object
 1   Стоимость привлечения, руб.  6 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 224.0+ bytes


Мы видим что в таблицах `customers.csv` и `cost_attraction.csv`:
- пропусков данных нет
- данные в полях в подходящих типах
- имена полей подходят нашим требованиям

# 2. Предобработка данных

### Проверка числовых значений:

Выполним проверку числовых значений в таблице `customers.csv` в поле `LTV, руб.` с целью исключения ошибок в данных. Отсортируем значения, так как при сортировке показываются ошибки в данных.

In [127]:
# отсортируем значения в df по полю `LTV, руб.` по возростанию
df_sorted = df_customers.sort_values(by='LTV, руб.', ascending=True)
# первые 5 элементов
df_sorted.head()

,id клиента,Канал привлечения,"LTV, руб."
36,37,Реклама у блогеров,0.0
1718,1719,Таргетная реклама,0.0
1717,1718,Реклама у блогеров,0.0
930,931,Реклама у блогеров,0.0
73,74,Контекстная реклама,0.0


In [128]:
# последние 5 элементов
df_sorted.tail()

,id клиента,Канал привлечения,"LTV, руб."
1457,1458,Таргетная реклама,847.0
597,598,Таргетная реклама,851.0
589,590,Таргетная реклама,854.0
331,332,Таргетная реклама,857.0
234,235,Таргетная реклама,858.0


Мы видим, что в строке с минимальным значением `LTV, руб.` равно `-100000` - это ошибка в данных и удалим данную строку. 

In [129]:
# пересохраним в df с данными по клиентам все строки, кроме строки с ошибочным значением
df_customers = df_customers[df_customers['LTV, руб.'] != -100000]
# выполним повторную сортировку для проверки
df_customers = df_customers.sort_values(by='LTV, руб.', ascending=True)
# выведем повторно для проверки удаления ошибочного значения
df_customers.head()

,id клиента,Канал привлечения,"LTV, руб."
36,37,Реклама у блогеров,0.0
1718,1719,Таргетная реклама,0.0
1717,1718,Реклама у блогеров,0.0
930,931,Реклама у блогеров,0.0
73,74,Контекстная реклама,0.0


### Проверка на дубликаты:

Проверим на уникальность пользователей в поле `id клиента` таблицы `customers.csv`

In [130]:
# кол-во уникальных пользователей
df_customers['id клиента'].nunique()

1800

Все 1800 пользователей уникальные.

Проверим на уникальность названия каналов в поле `канал привлечения` таблицы `customers.csv`

In [131]:
# кол-во уникальных пользователей
df_customers['Канал привлечения'].nunique()

6

Выведем уникальные названия каналов

In [132]:
# уникальные значения в поле
df_customers['Канал привлечения'].unique()

array(['Реклама у блогеров', 'Таргетная реклама', 'Контекстная реклама',
       'Нативная реклама', 'e-mail рассылки', 'Реклама в новостных СМИ'],
      dtype=object)

Все названия каналов уникальные.

# 3. Расчёты

### 3.1 Создаём сводную таблицу по каналам привлечения клиентов с добавлением двух полей:
1. Средний LTV на канал в руб.
2. Кол-во клипентов, пришедших из канала

In [137]:
# создаём сводную таблицу
df_agg = df_customers.groupby(['Канал привлечения',]).agg(
	mean = ('LTV, руб.', lambda x: round(x.mean(), 0)),  # Средний LTV на канал в руб. Подсчёт с округлением
	count = ('LTV, руб.', 'count'),                      # Кол-во клипентов, пришедших из канала. Подсчёт без округления
	).reset_index()

# переименовываем названия столбцов
df_agg.rename(columns={'mean': 'Средний LTV, руб.', 'count': 'Кол-во клиентов'}, inplace=True)
df_agg

,Канал привлечения,"Средний LTV, руб.",Кол-во клиентов
0,e-mail рассылки,380.0,112
1,Контекстная реклама,279.0,141
2,Нативная реклама,359.0,253
3,Реклама в новостных СМИ,375.0,301
4,Реклама у блогеров,350.0,691
5,Таргетная реклама,346.0,302


### 3.2 К таблице с общими затратами присоединяем нашу сводную таблицу:

In [138]:
# соединяем таблицы по полю `Канал привлечения`
df_group = pd.merge(df_cost_attr, df_agg, on='Канал привлечения', how='inner')
df_group

,Канал привлечения,"Стоимость привлечения, руб.","Средний LTV, руб.",Кол-во клиентов
0,Таргетная реклама,100000,346.0,302
1,e-mail рассылки,25000,380.0,112
2,Реклама у блогеров,120000,350.0,691
3,Контекстная реклама,90000,279.0,141
4,Нативная реклама,70000,359.0,253
5,Реклама в новостных СМИ,60000,375.0,301


### 3.3 Считаем `CAC` (сколько денег компания тратит на привлечение одного клиента.), поделив общие издержки, на количество пользователей:

In [139]:
# добавляем новый столбец как результат деления полей `Стоимость привлечения, руб.` на `Кол-во клиентов` с округлением
df_group = df_group.assign(cac = round(df_group['Стоимость привлечения, руб.'] / df_group['Кол-во клиентов'], 1))
df_group

,Канал привлечения,"Стоимость привлечения, руб.","Средний LTV, руб.",Кол-во клиентов,cac
0,Таргетная реклама,100000,346.0,302,331.1
1,e-mail рассылки,25000,380.0,112,223.2
2,Реклама у блогеров,120000,350.0,691,173.7
3,Контекстная реклама,90000,279.0,141,638.3
4,Нативная реклама,70000,359.0,253,276.7
5,Реклама в новостных СМИ,60000,375.0,301,199.3


### 3.4. Вычитаем из LTV средние издержки и строим итоговую таблицу:

In [140]:
# добавляем новый столбец как результат деления полей `Стоимость привлечения, руб.` на `Кол-во клиентов`
df_group = df_group.assign(ltv_cac = round(df_group['Средний LTV, руб.'] - df_group['cac'], 1))

# переименовываем названия столбцов
df_group.rename(columns={'cac': 'CAC, руб.', 'ltv_cac': 'LTV-CAC, руб.'}, inplace=True)
df_group

,Канал привлечения,"Стоимость привлечения, руб.","Средний LTV, руб.",Кол-во клиентов,"CAC, руб.","LTV-CAC, руб."
0,Таргетная реклама,100000,346.0,302,331.1,14.9
1,e-mail рассылки,25000,380.0,112,223.2,156.8
2,Реклама у блогеров,120000,350.0,691,173.7,176.3
3,Контекстная реклама,90000,279.0,141,638.3,-359.3
4,Нативная реклама,70000,359.0,253,276.7,82.3
5,Реклама в новостных СМИ,60000,375.0,301,199.3,175.7


### Диаграммы:

# 4. Результаты расчётов

... описать результаты расчётов ...